# EXTRACT

In [3]:
import pandas as pd
import openpyxl
import awswrangler as awr

xlsx = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_desativadas.xlsx"
df_desativadas = pd.read_excel(xlsx, engine = 'openpyxl')


In [8]:
df_desativadas.drop_duplicates(subset='chassi').shape[0]

972

In [5]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\sql\others_query.sql"
with open(query_path, "r") as file:
    query = file.read()

df_conferencia = awr.athena.read_sql_query(query, database='silver')
df_conferencia.head()  

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,consultor,status,...,empresa,coverage,beneficio,categoria,tipo_categoria,data_ativacao_beneficio,status_beneficio,data_atualizacao,consultor_ativo,id_beneficio
0,CYB0A45,9BM6931272B316506,1177623,1177623,<NA>,3207,4251,You Safer c,Matheus Junior dos Santos,NOVO,...,Tag,725565,APP,PADRÃO,PADRÃO,2025-09-26,NOVO,2025-09-26,S,48
1,OGQ8G19,9AA07123GCC112805,7621,0,7621,2294,3827,UNIDADE RIO VERDE,Gabriel Soares,ATIVO,...,Tag,717011,Casco PT (R/SR),GRANELEIRO,PADRÃO,2025-09-22,ATIVO,2025-09-22,S,35
2,OGS7I58,9BVAG20D1CE787691,939972,939972,<NA>,2294,3827,UNIDADE RIO VERDE,Gabriel Soares,ATIVO,...,Tag,717008,Assistência 24 Horas,PESADOS - CAVALO MECÂNICO,ATÉ 15 ANOS,2025-09-22,ATIVO,2025-09-22,S,41
3,SCY8F77,94BL0572PRV007818,6039,0,6039,2294,2853,UNIDADE RIO VERDE,Luiz Andre Senna Muniz Barreto Junior,ATIVO,...,Tag,708027,Assistência a Reparos (R/SR),GRANELEIRO,PADRÃO - 4%,2025-09-08,ATIVO,2025-09-08,S,38
4,SCY8F37,94BA0952PRV007817,6040,0,6040,2294,2853,UNIDADE RIO VERDE,Luiz Andre Senna Muniz Barreto Junior,ATIVO,...,Tag,708026,Casco PT (R/SR),GRANELEIRO,PADRÃO,2025-09-08,ATIVO,2025-09-08,S,35


In [12]:
df_conferencia_sorted = df_conferencia.sort_values(by='data_ativacao', ascending=False)
df_conferencia_sorted.drop_duplicates(subset='chassi', keep='first', inplace=True)

In [15]:
df_conferencia_sorted.drop_duplicates(subset='chassi', inplace=True)

In [16]:
df_conferencia_sorted.shape[0]

108385

In [17]:
df_conferencia_sorted.columns

Index(['placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria',
       'matricula', 'conjunto', 'unidade', 'consultor', 'status', 'cliente',
       'data_registro', 'data_ativacao', 'suporte', 'data_filtro', 'empresa',
       'coverage', 'beneficio', 'categoria', 'tipo_categoria',
       'data_ativacao_beneficio', 'status_beneficio', 'data_atualizacao',
       'consultor_ativo', 'id_beneficio'],
      dtype='object')

In [18]:

df_desativadas_status = df_desativadas.merge(
    df_conferencia_sorted[['chassi', 'status_beneficio']],
    on='chassi',
    how='left'
)


In [19]:
df_desativadas_status.shape[0]

972

In [22]:
df_desativadas_status.columns

Index(['placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria',
       'matricula', 'conjunto', 'unidade', 'consultor', 'status_beneficio_x',
       'cliente', 'data_registro', 'data_ativacao', 'data_ativacao_beneficio',
       'suporte', 'data_filtro', 'empresa', 'migration_from',
       'status_beneficio_y'],
      dtype='object')

In [23]:
df_desativadas_status['status_beneficio_y'].value_counts(dropna=False)

status_beneficio_y
ATIVO          806
CANCELADO      145
NOVO            19
<NA>             1
SUBSTITUÍDO      1
Name: count, dtype: Int64